In [444]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib


import os,sys
from importlib import reload
import copy
from collections import namedtuple

sys.path.append("../tracker")

In [445]:
import kalmanfilter as KF
import utilities as Util
import datatypes
import vertexfinder as VF

reload(VF)
reload(TF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [474]:
tracks=joblib.load("tracks_example.joblib")
# tracks

In [475]:
vf = VF.VertexFitter(debug=True)
vertices = vf.run(tracks)
# %timeit -n 2 vf.run(tracks)

VertexSeed(x0=-429.1761485809954, y0=11458.743993315009, z0=13175.858970818914, t0=70.86179389996407, cov=0, chi2=15.058267568254465, dist=226.27688130352624, Ntracks=0, trackind1=0, trackind2=3)
VertexSeed(x0=-916.6426779019071, y0=9298.170775219201, z0=11843.262543428742, t0=2.817481274571789, cov=0, chi2=0.8449106967371279, dist=16.868818487332014, Ntracks=3, trackind1=1, trackind2=6)
--- New seed for vertex --- 
  Seed VertexSeed(x0=-916.6426779019071, y0=9298.170775219201, z0=11843.262543428742, t0=2.817481274571789, cov=0, chi2=0.8449106967371279, dist=16.868818487332014, Ntracks=3, trackind1=1, trackind2=6)
  * Track [5] removed from vertex fit. Fit valid: True; vertex chi2_r 31.79; vertex chi2 increment 158.31
  * Track [3] removed from vertex fit. Fit valid: True; vertex chi2_r 23.96; vertex chi2 increment 119.15
 -> Track [4] added to vertex. Vertex chi2_r 3.77; vertex chi2 increment 0.00. Track: (-949.4511922438071, 9894.0, 13096.803856255683, 45.628423533705366, 0.070791936

In [473]:
0.01*(11923.145316440205-7000-4950), 0.01*(-998.8637214423652), 0.01*(9419.048555922185-8547) 

(-0.2685468355979538, -9.988637214423653, 8.720485559221853)

In [476]:
print(len(vertices))
print(len(vertices[0].tracks))
print(vertices[0])

2
3
Vertex(x0=-998.7019943955092, y0=9419.927995448612, z0=11923.441858283331, t0=6.473822458657651, cov=array([[ 19.54017141,  -9.16181912, -13.54570357,  -0.60782223],
       [ -9.16181912,  21.47617524,  31.75241728,   1.4247931 ],
       [-13.54570357,  31.75241728,  69.15967667,   2.10654944],
       [ -0.60782223,   1.4247931 ,   2.10654944,   4.73892007]]), chi2=18.850297384053682, tracks=[Track(x0=-1365.0108087953533, y0=9894.0, z0=12258.551342056233, t0=28.89149054310636, Ax=-0.7441830132248165, Ay=1, Az=0.691579950037166, At=0.0439238249877844, cov=array([[ 4.12807664e+00,  0.00000000e+00,  0.00000000e+00,
        -2.02527097e-02,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.68043024e+00,  0.00000000e+00,
         0.00000000e+00, -1.02541541e-02,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  7.00000000e-01,
         0.00000000e+00,  0.00000000e+00, -3.67647059e-03],
       [-2.02527097e-02,  0.00000000e+00,  0.00000000e+00,
         1.24619542

In [ ]:
def root_hits_extractor_tracks(Tree, entry):
    Tree.GetEntry(entry)
    c2list = lambda x: np.array([x.at(i) for i in range(x.size())])

    Digi_layer = c2list(Tree.Digi_layer_id)
    # Only select hits in tracking layer
    mask = (Digi_layer>1) & (Digi_layer<=5) 

    Digi_x = c2list(Tree.Digi_x)[mask]
    Digi_y = c2list(Tree.Digi_y)[mask]
    Digi_z = c2list(Tree.Digi_z)[mask]
    Digi_t = c2list(Tree.Digi_time)[mask]
    Digi_track_id = c2list(Tree.Digi_track_id)[mask]
    Digi_layer = Digi_layer[mask]
    Hits = Util.hit.make_hits(Digi_x, Digi_y, Digi_z, Digi_t, Digi_layer)

    track_ids = np.unique(Digi_track_id)
    track_hits  = {}
    for trackid in track_ids:
        mask = Digi_track_id==trackid
        hit_group = []
        for i in np.flatnonzero(mask):
            hit_group.append(Hits[i])
        track_hits[int(trackid)] = hit_group

    return track_hits